In [1]:
import pandas as pd

In [16]:
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize, sent_tokenize
import gensim
from gensim import corpora

In [33]:
from sklearn.feature_extraction.text import TfidfVectorizer
import numpy as np

In [27]:
add_stop = ['local', 'law', 'chair', 'land', 'comittee','application','oversight','committee','application','number','land use',
            'resolution','communication']

In [28]:
def preprocess_data(documents):
    stop_words = set(stopwords.words('english'))
    stop_words.update(add_stop)

    tokenized_docs = [word_tokenize(doc.lower()) for doc in documents]
    processed_docs = [[word for word in doc if word not in stop_words and word.isalpha()] for doc in tokenized_docs]
    return processed_docs

#### Impot and clean legislation tables

In [9]:
try:
    leg_2024 = pd.read_csv('data/cy2024csv.csv', encoding='ISO-8859-1')
except UnicodeDecodeError:
    try:
        leg_2024 = pd.read_csv('data/cy2024csv.csv', encoding='cp1252')
    except UnicodeDecodeError:
        pass

In [11]:
try:
    leg_2023 = pd.read_csv('data/cy2023csv.csv', encoding='ISO-8859-1')
except UnicodeDecodeError:
    try:
        leg_2023 = pd.read_csv('data/cy2023csv.csv', encoding='cp1252')
    except UnicodeDecodeError:
        pass

In [18]:
all_leg = pd.concat([leg_2023, leg_2024])

In [29]:
all_leg['processed_text'] = all_leg['Title'].apply(lambda x: preprocess_data([x])[0])

In [31]:
all_leg = all_leg[['Type','Status','Committee','Prime Sponsor','processed_text']]
all_leg = all_leg.rename(columns={'Prime Sponsor':'prime_sponsor'})
all_leg = all_leg.dropna()

#### tf-idf

In [36]:
def join_tokens(token_list):
    return ' '.join(token_list)

In [37]:
all_leg['joined_text'] = all_leg['processed_text'].apply(join_tokens)

In [38]:
grouped_df = all_leg.groupby('prime_sponsor')['joined_text'].apply(' '.join).reset_index()

In [34]:
tfidf = TfidfVectorizer()